In [ ]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Parameter
from qiskit.quantum_info import partial_trace, entropy
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.algorithms.optimizers import SPSA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns

class CustomFeatureMap:
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits

    def create_circuit(self, x):
        qc = QuantumCircuit(self.n_qubits)
        for i in range(self.n_qubits):
            qc.ry(x[i], i)
        for i in range(self.n_qubits-1):
            qc.cx(i, i+1)
        for i in range(self.n_qubits):
            qc.rz(x[i]**2, i)
        return qc

class CustomAnsatz:
    def __init__(self, n_qubits, depth=2):
        self.n_qubits = n_qubits
        self.depth = depth
        self.parameters = []
        for d in range(depth):
            layer_params = [Parameter(f'θ_{d}_{i}') for i in range(n_qubits * 3)]
            self.parameters.extend(layer_params)

    def create_circuit(self):
        qc = QuantumCircuit(self.n_qubits)
        param_idx = 0
        for d in range(self.depth):
            for i in range(self.n_qubits):
                qc.rx(self.parameters[param_idx], i)
                param_idx += 1
                qc.ry(self.parameters[param_idx], i)
                param_idx += 1
                qc.rz(self.parameters[param_idx], i)
                param_idx += 1
            if d < self.depth - 1:
                for i in range(self.n_qubits-1):
                    qc.cx(i, i+1)
                qc.cx(self.n_qubits-1, 0)
        return qc

class VariationalQuantumClassifier:
    def __init__(self, n_qubits, feature_map=None, ansatz=None, noise_model=None):
        self.n_qubits = n_qubits
        self.feature_map = feature_map or CustomFeatureMap(n_qubits)
        self.ansatz = ansatz or CustomAnsatz(n_qubits)
        self.parameters = None
        self.simulator = QasmSimulator()
        if noise_model:
            self.simulator.set_options(noise_model=noise_model)

    def create_circuit(self, x):
        qc = QuantumCircuit(self.n_qubits, 1)
        qc.compose(self.feature_map.create_circuit(x), inplace=True)
        var_circuit = self.ansatz.create_circuit()
        qc.compose(var_circuit, inplace=True)
        qc.measure_all()
        return qc

    def compute_expressibility(self, n_samples=1000):
        random_params1 = np.random.random(len(self.ansatz.parameters))
        random_params2 = np.random.random(len(self.ansatz.parameters))
        circuit1 = self.ansatz.create_circuit()
        circuit2 = self.ansatz.create_circuit()
        bound_circuit1 = circuit1.bind_parameters({p: v for p, v in zip(self.ansatz.parameters, random_params1)})
        bound_circuit2 = circuit2.bind_parameters({p: v for p, v in zip(self.ansatz.parameters, random_params2)})
        return np.mean(np.abs(random_params1 - random_params2))

    def compute_entangling_capacity(self):
        qc = self.ansatz.create_circuit()
        random_params = np.random.random(len(self.ansatz.parameters))
        bound_circuit = qc.bind_parameters({p: v for p, v in zip(self.ansatz.parameters, random_params)})
        return entropy(partial_trace(bound_circuit, [1]))

    def fit(self, X, y, optimizer=None):
        optimizer = optimizer or SPSA(maxiter=100)

        def objective_function(params):
            cost = 0
            for x_i, y_i in zip(X, y):
                circuit = self.create_circuit(x_i)
                bound_circuit = circuit.bind_parameters({p: v for p, v in zip(self.ansatz.parameters, params)})
                job = self.simulator.run(bound_circuit, shots=1000)
                counts = job.result().get_counts()
                prob_0 = counts.get('0', 0) / 1000
                prob_1 = counts.get('1', 0) / 1000
                if y_i == 0:
                    cost += (1 - prob_0)**2
                else:
                    cost += (1 - prob_1)**2
            return cost / len(X)

        initial_params = np.random.random(len(self.ansatz.parameters))
        self.parameters = optimizer.optimize(len(initial_params), objective_function, initial_point=initial_params)[0]

    def predict(self, X):
        predictions = []
        for x in X:
            circuit = self.create_circuit(x)
            bound_circuit = circuit.bind_parameters({p: v for p, v in zip(self.ansatz.parameters, self.parameters)})
            job = self.simulator.run(bound_circuit, shots=1000)
            counts = job.result().get_counts()
            pred = 1 if counts.get('1', 0) > counts.get('0', 0) else 0
            predictions.append(pred)
        return np.array(predictions)

from qiskit.providers.aer.noise import depolarizing_error, QuantumError

def create_noise_model():
    noise_model = NoiseModel()
    # Create a depolarizing error for single-qubit gates
    error_1_qubit = depolarizing_error(0.01, 1)
    # Add the error to single-qubit gates
    noise_model.add_all_qubit_quantum_error(error_1_qubit, ['u1', 'u2', 'u3'])
    return noise_model

def plot_results(metrics_dict, title):
    plt.figure(figsize=(10, 6))
    plt.bar(metrics_dict.keys(), metrics_dict.values())
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{title.lower().replace(' ', '_')}.png")
    plt.close()

def main():
    # Load and preprocess data
    data = pd.read_csv('dataset.csv')
    X = data.drop('quality', axis=1).values
    y = data['quality'].values

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize models
    vqc_clean = VariationalQuantumClassifier(n_qubits=X_train.shape[1])
    vqc_noisy = VariationalQuantumClassifier(n_qubits=X_train.shape[1], noise_model=create_noise_model())
    svm_classifier = SVC(kernel='rbf')

    # Train and evaluate clean VQC
    print("Training clean VQC...")
    vqc_clean.fit(X_train_scaled, y_train)
    clean_pred = vqc_clean.predict(X_test_scaled)
    clean_metrics = {
        'accuracy': accuracy_score(y_test, clean_pred),
        'expressibility': vqc_clean.compute_expressibility(),
        'entangling_capacity': vqc_clean.compute_entangling_capacity()
    }

    # Train and evaluate noisy VQC
    print("Training noisy VQC...")
    vqc_noisy.fit(X_train_scaled, y_train)
    noisy_pred = vqc_noisy.predict(X_test_scaled)
    noisy_metrics = {
        'accuracy': accuracy_score(y_test, noisy_pred),
        'expressibility': vqc_noisy.compute_expressibility(),
        'entangling_capacity': vqc_noisy.compute_entangling_capacity()
    }

    # Train and evaluate SVM
    print("Training SVM...")
    svm_classifier.fit(X_train_scaled, y_train)
    svm_pred = svm_classifier.predict(X_test_scaled)
    svm_metrics = {
        'accuracy': accuracy_score(y_test, svm_pred)
    }

    # Plot results
    plot_results(clean_metrics, "Clean VQC Metrics")
    plot_results(noisy_metrics, "Noisy VQC Metrics")

    # Print comparison
    print("\nModel Comparison:")
    print(f"Clean VQC Accuracy: {clean_metrics['accuracy']:.4f}")
    print(f"Noisy VQC Accuracy: {noisy_metrics['accuracy']:.4f}")
    print(f"SVM Accuracy: {svm_metrics['accuracy']:.4f}")

if __name__ == "__main__":
    main()